# スイープ ジョブを使用してハイパーパラメーターを調整する

ハイパーパラメーター (トレーニングに影響を与えるが、トレーニングデータ自体からは決定できないパラメーター値) を必要とする機械学習アルゴリズムは多数あります。 たとえば、ロジスティック回帰モデルをトレーニングする場合、正規化率ハイパーパラメーターを使用してモデルのバイアスに対抗できます。畳み込みニューラル ネットワークをトレーニングする場合、学習率やバッチ サイズなどのハイパーパラメーターを使用して、重みの調整方法とミニバッチで処理されるデータ項目の数をそれぞれ制御できます。 ハイパーパラメーター値の選択は、トレーニング済みモデルのパフォーマンスやトレーニングにかかる時間に大きく影響する可能性があります。多くの場合、最適なソリューションを見つけるには複数の組み合わせを試す必要があります。 

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、Studio の右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## トレーニング スクリプトの作成
ハイパーパラメーターの調整は、機械学習モデルをトレーニングするが、入力パラメーターを変更する場合に最適です。 アルゴリズムのハイパーパラメーターのいずれかを表す入力パラメーターを必要とするトレーニング スクリプトを作成する必要があります。

次のセルを実行して、**src** フォルダーとトレーニング スクリプトを作成します。

トレーニング スクリプトには次の 2 つの入力パラメーターが必要であることに注意してください。

- `--training_data` には文字列が必要です。 登録されたデータ資産へのパスを入力トレーニング データとして指定します。
- `--reg_rate` には数値が必要ですが、既定値は `0.01` です。 ハイパーパラメーターの調整には、この入力パラメーターを使用します。

In [ ]:
## コマンド ジョブを構成して実行する

以下のセルを実行して、糖尿病を予測する分類モデルをトレーニングします。 モデルは、**src** フォルダーにある **train\.py** スクリプトを実行してトレーニングされます。 登録された `diabetes-data` データ資産がトレーニング データとして使用されます。 

- `code`: 実行するスクリプトを含むフォルダーを指定します。
- `command`: 正確に実行する内容を指定します。
- `environment`: コマンドを実行する前にコンピューティングにインストールする必要があるパッケージを指定します。
- `compute`: コマンドの実行に使用するコンピューティングを指定します。
- `display_name`: 個々のジョブの名前。
- `experiment_name`: ジョブが属する実験の名前。

コマンド ジョブではトレーニング スクリプトが 1 回だけ実行され、正則化率は `0.1` であることに注意してください。 ハイパーパラメーターを調整するためにスイープ ジョブを実行する前に、コマンド ジョブでスクリプトが期待どおりに動作するかどうかをテストすることをお勧めします。

In [ ]:
## 検索空間を定義する

コマンド ジョブが正常に完了したら、スイープ ジョブを構成して実行できます。 

最初に、ハイパーパラメーターの検索空間を指定する必要があります。 それぞれ異なる正則化率 (`0.01`、`0.1`、または `1`) の 3 つのモデルをトレーニングするために、`Choice` ハイパーパラメーターを使用して検索空間を定義できます。 

## スイープ ジョブを構成して送信する

スイープ関数を使用して、トレーニング スクリプトでハイパーパラメーターの調整を行います。 スイープ ジョブを構成するには、以下を構成する必要があります。

- `compute`: ジョブを実行するコンピューティング先の名前。
- `sampling_algorithm`: 検索空間に対して使用するハイパーパラメーター サンプリング アルゴリズム。 使用できる値は、`random`、`grid`、`bayesian` です。
- `primary_metric`: 各試用版ジョブによって報告されるプライマリ メトリックの名前。 メトリックは、同じ対応するメトリック名を持つ `mlflow.log_metric()` を使用して、ユーザーのトレーニング スクリプトに記録する必要があります。
- `goal`: `primary_metric` の最適化の目標。 指定できる値は、`maximize` と `minimize` です。
- `limits`: スイープ ジョブの制限。 たとえば、トレーニングする試用版やモデルの最大数です。

コマンド ジョブはスイープ ジョブのベースとして使用されることに注意してください。 コマンド ジョブの構成は、スイープ ジョブで再利用されます。

In [ ]:
次のセルを実行して、スイープ ジョブを送信します。

In [ ]:
ジョブが完了したら、ジョブの概要に移動します。 **[試用版]** タブには、トレーニングされたすべてのモデルと、試した正則化率の値ごとに `Accuracy` スコアがどのように異なるかが示されます。

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.